# Keras Image Classifier (Using ImageDataGenerator and Transfer Learning)

The code in this notebook uses the convolutional layers of the Inception model with pretrained weights loaded and then adds fully connected layers to be trained for this specific task.  Training for the convolutional layers is frozen.

## Load Inception Model

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (300, 300, 3), 
                                include_top = False, 
                                weights = "imagenet")

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Inception model summary
pre_trained_model.summary()

## Create Fully Connected Layers and Compile Model

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

## Data Preproc

Training and evaluation data need to be placed in separate folders with paths noted below; within each of those folders all examples of each class should be placed within their own subfolders e.g. for a dogs vs cats image classifier, all dog training data should be placed in ./Cats_Dogs_Images/Train/Dogs and all cat training data should be placed in ./Cats_Dogs_Images/Train/Cats.  Only the path for the parent folder needs to be noted below; ImageDataGenerator picks up the two subfolders itself and interprets them as being the two classes.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images rescaled (normalized) by 1./255; training images augmented
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './Cats_Dogs_Images/Train/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        './Cats_Dogs_Images/Test/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

## Fit/Train Model

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=152,  
      epochs=2,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=38)

## Plot Learning Curves

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Model Summary

In [ ]:
model.summary()

## Make Predictions

In [ ]:
from keras.preprocessing import image
import numpy as np

path = './Cats_Dogs_Images/dog_test.jpg'
img = image.load_img(path, target_size=(300, 300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print("Dog")
else:
    print("Cat")
 